In [ ]:
# Install necessary libraries
!pip install boto3 sagemaker pandas scikit-learn

# Import necessary libraries
import boto3
import pandas as pd
import io
import json
from datetime import datetime

# Define the SageMaker endpoint name and region
endpoint_name = 'simple-linear-regression-endpoint'
region_name = 'us-east-1'
s3_bucket = 'justin-automation-output'  # Replace with your S3 bucket name for inference results

# Function to invoke the SageMaker endpoint
def invoke_endpoint(payload):
    client = boto3.client('sagemaker-runtime', region_name=region_name)
    response = client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(payload),
        ContentType='application/json'
    )
    result = json.loads(response['Body'].read().decode())
    return result

# Test the function with a sample payload
payload = {"data": [[8.3252, 41.0, 6.984127, 1.023810, 322.0, 2.555556, 37.88, -122.23]]}  # Example input
result = invoke_endpoint(payload)
print("Inference result:", result)

# Convert the result to a pandas DataFrame
inference_data = {
    "input": [payload["data"]],
    "prediction": [result]
}
df = pd.DataFrame(inference_data)

# Save the DataFrame as a CSV file
csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=False)

# Upload the CSV file to S3
csv_file_name = f'inferences_{datetime.now().strftime("%Y%m%d%H%M%S")}.csv'
s3_client = boto3.client('s3', region_name=region_name)
s3_client.put_object(
    Bucket=s3_bucket,
    Key=csv_file_name,
    Body=csv_buffer.getvalue()
)

print(f"Inference results saved to S3 bucket '{s3_bucket}' with file name '{csv_file_name}'.")
